In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
32,34,Female,101.0,Strong,Delhi
78,11,Male,100.0,Mild,Bangalore
10,75,Female,NaN,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
36,38,Female,101.0,Mild,Bangalore
...,...,...,...,...,...
35,82,Female,102.0,Strong,Bangalore
4,65,Female,101.0,Mild,Mumbai
53,83,Male,98.0,Mild,Delhi
77,8,Female,101.0,Mild,Kolkata


In [17]:
X_train.isna().sum()

age       0
gender    0
fever     8
cough     0
city      0
dtype: int64

# simple imputing, ordinary encoding , ordinlencoding

## SimpleImputer - 

### we can use simple imputer to handle missing values in both categorical and numerical value

In [8]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

## Ordinalencoding 
categories will be given not oder


In [9]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

## OneHotEncoder-
### Note: OneHotEncoder can’t handle missing values, hence it is important to get rid of them before encoding

In [10]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [11]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [12]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [13]:
X_train_transformed

array([[ 34.        , 101.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 11.        , 100.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 75.        , 100.80555556,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 73.        , 103.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 38.        , 101.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 22.        ,  99.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 51.        , 104.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 13.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   1.        ],
       [  5.        ,  98.        ,   0.        ,   0.        ,
          0.    

## ColumnTransformer

#### ColumnTransformer enables us to transform a specified set of columns. It helps us to apply multiple transforms to multiple columns with a single fit() or fit_transform() statement. For example, we can mean impute the first column and one hot encode the second column of a data frame with a single fit() or fit_transform() statement. ColumnTransformer class can be imported from ‘sklearn.compose’ module as shown below.

In [32]:
from sklearn.compose import ColumnTransformer

In [95]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [97]:
transformer.fit_transform(X_train).shape

(80, 7)

In [99]:
transformer.transform(X_test).shape

(20, 7)

# Binning

### The Numerical Features can be converted to categorical by binnng that feature. This will help in generalising the model better.


In [ ]:
# we can use scikit-learn's Function Transfomer to use pandas qcut method for creating bins 
## Age Feature
# bins = [20, 30, 40, 50, 60, np.inf]
# labels = ['young_adult', 'adult', 'middle_aged', 'elderly', 'senior_citizen']
# age_transformer = FunctionTransformer(
#     pd.cut, kw_args={'bins': bins, 'labels': labels, 'retbins': False}
#     )
# input_df["age"] = age_transformer.fit_transform(input_df["age"])
# test_df["age"] = age_transformer.transform(test_df["age"])